In [12]:
! wget https://www.dropbox.com/s/t0fngc472gdi5bk/anime-recommendations-database.zip
! unzip anime-recommendations-database.zip
! ls

--2019-04-10 10:24:08--  https://www.dropbox.com/s/t0fngc472gdi5bk/anime-recommendations-database.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/t0fngc472gdi5bk/anime-recommendations-database.zip [following]
--2019-04-10 10:24:09--  https://www.dropbox.com/s/raw/t0fngc472gdi5bk/anime-recommendations-database.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca228545f1843f7c7719660e167.dl.dropboxusercontent.com/cd/0/inline/AeuobrM43chZiG44afRRsXibHNgZ-Ajf0zuoCequlHf8DsavQSqyiTMP-_RLN6NTDrbfwCNryejhw4KdLHmmjDOj1v60bSj_6ZbPjNC9pB-ILC5N1n0c4D7UEAQfKK_g3Lw/file# [following]
--2019-04-10 10:24:09--  https://uca228545f1843f7c7719660e167.dl.dropboxusercontent.com/cd/0/inline/AeuobrM43chZiG44afRRsXibHNgZ-Ajf0zuoCequlHf8D

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neighbors import NearestNeighbors
%matplotlib inline

In [14]:
df = pd.read_csv('anime.csv')
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [15]:
df['type'] = df['type'].fillna('None')
df['genre'] = df['genre'].fillna('None')
df['rating'] = df['rating'].fillna(df['rating'].median())
episode_ = df['episodes'].replace('Unknown', np.nan)
episode_ = episode_.fillna(episode_.median())
df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [16]:
type_ = pd.get_dummies(df['type'])
genre_ = df['genre'].str.get_dummies(sep=',')
genre_.head()

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X = pd.concat([genre_, type_, episode_, df['rating'], df['members']],axis=1)
X.head()

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,Movie,Music,None,ONA,OVA,Special,TV,episodes,rating,members
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,9.37,200630
1,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,64,9.26,793665
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,51,9.25,114262
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,24,9.17,673572
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,51,9.16,151266


In [0]:
scaled = MaxAbsScaler()
X = scaled.fit_transform(X)

In [19]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors
recommendations = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(X)
recommendations.kneighbors(X)

(array([[0.00000000e+00, 1.01506549e+00, 1.03095542e+00, ...,
         1.45773221e+00, 1.45854509e+00, 1.45930354e+00],
        [0.00000000e+00, 1.02227158e+00, 1.49378838e+00, ...,
         1.65016556e+00, 1.74844630e+00, 1.78151418e+00],
        [0.00000000e+00, 3.75894154e-02, 4.12559430e-02, ...,
         1.42104554e+00, 1.42183188e+00, 1.42808851e+00],
        ...,
        [0.00000000e+00, 1.66436926e-03, 1.68253094e-03, ...,
         8.07869678e-03, 1.01353307e-02, 1.11230973e-02],
        [0.00000000e+00, 1.00174940e-03, 1.10513608e-03, ...,
         4.01486740e-03, 5.00190620e-03, 5.03023042e-03],
        [0.00000000e+00, 1.00179958e+00, 1.41421365e+00, ...,
         1.41421400e+00, 1.41421404e+00, 1.41421417e+00]]),
 array([[    0,   208,  1494, ...,  5796,  5697,  5805],
        [    1,   200,   268, ...,  6868,   288,   374],
        [    2,     4,     9, ...,   216,   306,  1833],
        ...,
        [12291, 12238, 12237, ..., 12239, 12292, 12231],
        [12292, 12231, 1

In [0]:
anime_indices = recommendations.kneighbors(X)[1] 

In [0]:

def get_index(x):
    # gives index for the anime
    return df[df['name']==x].index.tolist()[0]

def get_recommend_item(anime):
    print('Here are 10 anime similar to', anime, ':' '\n')
    index = get_index(anime)
    
    # ignore first entry so as to not return the queried anime as similar to itself
    for i in anime_indices[index][1:]:
            print(df.iloc[i]['name'], 
                  '\n' 'Genre: ', df.iloc[i]['genre'],
                  '\n' 'Episode count: ', df.iloc[i]['episodes'],
                  '\n' 'Rating out of 10:', df.iloc[i]['rating'], '\n')

In [22]:
get_recommend_item('Death Note')


Here are 10 anime similar to Death Note :

Higurashi no Naku Koro ni Kai 
Genre:  Mystery, Psychological, Supernatural, Thriller 
Episode count:  24 
Rating out of 10: 8.41 

Death Note Rewrite 
Genre:  Mystery, Police, Psychological, Supernatural, Thriller 
Episode count:  2 
Rating out of 10: 7.84 

Jigoku Shoujo Mitsuganae 
Genre:  Mystery, Psychological, Supernatural 
Episode count:  26 
Rating out of 10: 7.81 

Yakushiji Ryouko no Kaiki Jikenbo 
Genre:  Mystery, Police, Supernatural 
Episode count:  13 
Rating out of 10: 7.19 

Saint Luminous Jogakuin 
Genre:  Mystery, Psychological, Supernatural 
Episode count:  13 
Rating out of 10: 6.17 

Boku dake ga Inai Machi 
Genre:  Mystery, Psychological, Seinen, Supernatural 
Episode count:  12 
Rating out of 10: 8.65 

Shiki 
Genre:  Mystery, Supernatural, Thriller, Vampire 
Episode count:  22 
Rating out of 10: 7.99 

Mousou Dairinin 
Genre:  Drama, Mystery, Police, Psychological, Supernatural, Thriller 
Episode count:  13 
Rating out 